# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG =  or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 100

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
 'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
 'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[1.972 1.97  2.018 1.959 1.973 1.987 1.969 1.977 2.    1.976]
 [2.992 2.902 3.055 3.066 3.024 2.916 3.031 2.897 3.018 2.998]
 [3.891 4.124 4.143 3.963 3.98  3.94  3.928 4.061 3.985 3.993]] [[0.993216 0.9431   1.079676 0.959319 0.984271 0.966831 0.898039 0.964471
  0.944    0.975424]
 [1.967936 1.828396 2.053975 2.077644 1.967424 2.062944 2.022039 1.996391
  1.969676 1.973996]
 [2.849119 3.194624 2.956551 3.071631 3.2116   3.0044   2.996816 3.167279
  2.974775 3.004951]]
Mean and variance endomwnets:
[[2.017 2.012 2.067 2.012 1.981 1.968 1.999 2.024 2.001 1.968]
 [3.08  3.056 3.085 3.027 3.048 2.998 3.042 3.045 2.957 3.025]
 [4.045 3.95  3.947 4.002 4.001 3.95  3.964 4.011 3.956 4.087]] [[1.028711 1.101856 1.040511 1.039856 0.930639 0.920976 1.070999 0.973424
  0.968999 0.972976]
 [1.9896   2.096864 2.007775 1.918271 2.001696 1.933996 2.010236 2.064975
  2.175151 1.956375]
 [3.042975 2.9795   2.814191 3.095996 2.960999 2.8295   3.090704 3.008879
  2.962064

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = {'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG), \
                      'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG), \
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG), \
                      'Budget' : lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

FixedThreshold
Lower Solutions:
[[0.      0.15789]
 [0.      0.15789]
 [0.19738 0.     ]]
Out of Budget!
Writing to file data.csv
Guardrail-0.5
Lower and Upper Solutions:
[[0.      0.15856]
 [0.      0.15856]
 [0.19658 0.     ]]
[[0.      0.23189]
 [0.      0.23189]
 [0.28749 0.     ]]
Out of Budget!
Writing to file data.csv
Guardrail-0.3
Lower and Upper Solutions:
[[ 0.       0.15717]
 [-0.       0.15717]
 [ 0.19835  0.     ]]
[[0.      0.31508]
 [0.      0.31508]
 [0.39764 0.     ]]
Out of Budget!
Writing to file data.csv
Guardrail-0.25
Lower and Upper Solutions:
[[0.      0.15772]
 [0.      0.15772]
 [0.19683 0.     ]]
[[0.      0.36038]
 [0.      0.36038]
 [0.44975 0.     ]]
Out of Budget!
Writing to file data.csv
traj len 1000
times out of budget 1
num_iter 100
traj len 1000
times out of budget 1
num_iter 100
traj len 1000
times out of budget 1
num_iter 100
traj len 1000
times out of budget 1
num_iter 100
        Algorithm  Reward      Time     Space  Efficiency  Hindsight Envy  \

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)